In [ ]:
library(dplyr)
library(OmnipathR)
library(reshape2)
library(igraph)

# Plotting networks

In [ ]:
g <- igraph::static.power.law.game(20, 100, 2, 2)
plot(g)

In [ ]:
g <- igraph::erdos.renyi.game(20, 20, "gnm", directed=TRUE)
plot(g)

# Omnipath

In [ ]:
omnipath = import_omnipath_interactions(from_cache_file = NULL,
                                        filter_databases = get_interaction_resources(), 
                                        select_organism = 9606)
omnipath

In [ ]:
filter_by_references <- 0

omnipath_pkn = omnipath %>% dplyr::filter( is_directed == 1 ) %>% 
dplyr::mutate_at( c("n_references"), ~as.numeric(as.character(.))) %>%
dplyr::filter( n_references >= filter_by_references ) %>%
dplyr::select( "source_genesymbol", "target_genesymbol", "consensus_stimulation", "consensus_inhibition" ) %>%
dplyr::mutate_at( c("consensus_stimulation", "consensus_inhibition"), ~as.numeric(as.character(.)))

omnipath_pkn_inh_upd = omnipath_pkn %>% melt() %>% 
filter(variable == "consensus_inhibition" & value == "1") %>% 
mutate(value = -1)
  
omnipath_pkn = omnipath_pkn %>% melt() %>% 
filter( variable == "consensus_stimulation" & value != "0" ) %>% 
rbind(omnipath_pkn_inh_upd) 

omnipath_pkn = omnipath_pkn %>% dplyr::select("source_genesymbol", "target_genesymbol", "value") %>% 
                              relocate("target_genesymbol", "value")
colnames(omnipath_pkn) = c("source", "effect", "target") 
omnipath_pkn

In [ ]:
generateTFList <- function (df = df, top = 50, access_idx = 1) 
{
  if (top == "all") {
    top <- nrow(df)
  }
  if (top > nrow(df)) {
    warning("Number of to TF's inserted exceeds the number of actual TF's in the\n            
            data frame. All the TF's will be considered.")
    top <- nrow(df)
  }
  ctrl <- intersect(x = access_idx, y = 1:ncol(df))
  if (length(ctrl) == 0) {
    stop("The indeces you inserted do not correspond to \n              
         the number of columns/samples")
  }
  returnList <- list()
  for (ii in 1:length(ctrl)) {
    tfThresh <- sort(x = abs(df[, ctrl[ii]]), decreasing = TRUE)[top]
    temp <- which(abs(df[, ctrl[ii]]) >= tfThresh)
    currDF <- matrix(data = , nrow = 1, ncol = top)
    colnames(currDF) <- rownames(df)[temp[1:top]]
    currDF[1, ] <- df[temp[1:top], ctrl[ii]]
    currDF <- as.data.frame(currDF)
    returnList[[length(returnList) + 1]] <- currDF
  }
  names(returnList) <- colnames(df)[ctrl]
  return(returnList)
}

In [ ]:
measurements = read.csv("Input/measurments.csv")
tf_activities_carnival <- data.frame(measurements, stringsAsFactors = F)
rownames(tf_activities_carnival) <- measurements$TF
tf_activities_carnival$TF <- NULL
tfList = generateTFList(tf_activities_carnival, top=50, access_idx = 1)
tfList

In [ ]:
runShuffler <- function(shufflingPercentage = seq(2, 50, 2), priorKnowledgeNetwork) {
  
  shuffledPkn <- lapply(shufflingPercentage, function(x, priorKnowledgeNetwork) {
    
    numberShuffledEdges <- x * length(priorKnowledgeNetwork[, 1]) / 100
    
    if (numberShuffledEdges == 0) {
      warning("0 shuffling percentage provided or number of edges to be shuffle is calculated to be 0. Returning an unchanged prior knowledge network.")
      return(priorKnowledgeNetwork)
    } else {
      #if rounded value is below 0, shuffle at least 2 edges
      numberShuffledEdges <- ifelse(round(numberShuffledEdges) > 0, round(numberShuffledEdges), 2) 
      
      # shuffle prior knowledge network in random order 
      priorKnowledgeNetwork <- priorKnowledgeNetwork[ sample(1:length(priorKnowledgeNetwork[, 1]), 
                                                             size = length(priorKnowledgeNetwork[, 1]), 
                                                             replace = F), ]
      
      edgesShuffledPkn <- priorKnowledgeNetwork
      # get randomly numberShuffledEdges numbers
      randomSampler <- sample(1:numberShuffledEdges, 
                              size = numberShuffledEdges, 
                              replace = F)
      
      edgesToRewire <- edgesShuffledPkn[ randomSampler, 1 ]
      # rewire edges
      edgesShuffledPkn[ 1:numberShuffledEdges,  1 ] <- edgesToRewire
      
      return(edgesShuffledPkn)  
    }
    
  }, priorKnowledgeNetwork=priorKnowledgeNetwork)
 
  return(shuffledPkn) 
}

callCarnivalWithShuffledPkn <- function( measurements, 
                                         perturbations, 
                                         priorKnowledgeNetworksShuffled, 
                                         cplexPath,
                                         outputFolder) {
  
  
  j <- 0
  for (i in priorKnowledgeNetworksShuffled) {
    
    # Identifying starting nodes 
    # N.B.Taken from transcript tutorial, simplest way to provide perturbations
    diffNodes = base::setdiff(i$source, i$target)
    initiators = base::data.frame(base::matrix(data = NaN, nrow = 1, ncol = length(iniMTX)), stringsAsFactors = F)
    colnames(initiators) = diffNodes
    print(initiators)
    
    j <- j + 1
    currentTime <- Sys.time()
    message(currentTime, ": Starting new run: ", j)
    resultOneRun <- runCARNIVAL(inputObj = initiators, 
                                measObj = measurements, 
                                netObj = i,
                                solver="cplex", 
                                solverPath=cplexPath)   
    currentTime <- Sys.time()
    message(currentTime, ": Finished run: ", j)
    collectedResults <- c(resultOneRun, collectedResults)
    
  }
  
  return(collectedResults)
}

In [ ]:
diffNodes = base::setdiff(omnipath_pkn$source, omnipath_pkn$target)
initiators = base::data.frame(base::matrix(data = NaN, nrow = 1, ncol = length(diffNodes)), stringsAsFactors = F)
colnames(initiators) = diffNodes
initiators 